In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
np.set_printoptions(precision=3, suppress=True)
from tensorflow import keras
from tensorflow.keras import layers
import array as arr

In [ ]:
from ROOT import TCanvas, TGraph, TFile, TH1F, TH2D
import numpy as np
import re
import scipy as sc
import matplotlib.pyplot as plt

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("current", graph.GetTitle())
            if searching != None:
                a = graph.GetTitle()[43:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                x_title = event.GetXaxis()
                if (str(x_title))[22:-1] != 'ns':
                    print('Units have changed')
                    break
                
    return pulsed_array, x, y

def collect_efield(files_tot, position):
    each_field = arr.array('f', [])
    for file in files_tot:
        field_values = arr.array('f', [])
        for f, p in zip(file, position):
            myFile = TFile.Open(f, "read")
            data = myFile.Get("ElectricFieldReader")
            detector = data.Get("detector1")
            efield = detector.Get("field1d_z")
            no_bins = efield.GetNbinsX()
            bin_size = 0.1/no_bins
            
            pos = p*10**(-3)
                
            placement = -0.05
            for bin in range(no_bins):  
                placement += bin_size
                if placement >= pos:
                    print(efield.GetBinContent(bin))
                    field_strength = arr.array('f', [efield.GetBinContent(bin)])
                    field_values = np.concatenate((field_values, field_strength))
                    break
                    
        each_field = np.concatenate((each_field, field_values))
    each_field = each_field.reshape((len(files_tot), len(position), 1))
        
    return each_field

def array_data(field_x, field_y, time_vals):
    field_data = arr.array('f', [])
    for x_data, y_data in zip(field_x, field_y):
        data_tot = arr.array('f', [])
        for x, y in zip(x_data, y_data):
            y_array = arr.array('f', [])
            for t in time_vals:
                if x[-1] < t:
                    y_array.append(0)
                    if len(y_array) == len(time_vals):
                        data_tot = np.concatenate((data_tot, y_array))
                
                else:
                    for i in range(len(x)):
                        if x[i] >= t and x[i-1] < t:
                            y_array.append(y[i])
                            if len(y_array) == len(time_vals):
                                data_tot = np.concatenate((data_tot, y_array))
                            break
                        else:
                            continue
    
        field_data = np.concatenate((field_data, data_tot))
    field_data = field_data.reshape((len(field_x), len(x_data), len(time_vals)))

    return field_data

files1 = ["root_files/data_10ps_n40.root", "root_files/data_10ps_n30.root", "root_files/data_10ps_n20.root", "root_files/data_10ps_n10.root","root_files/data_10ps_0.root", "root_files/data_10ps_p10.root", "root_files/data_10ps_p20.root", "root_files/data_10ps_p30.root", "root_files/data_10ps_p40.root"] 
files2 = ["root_files/less_overbias_n40.root", "root_files/less_overbias_n30.root", "root_files/less_overbias_n20.root", "root_files/less_overboas_n10.root", "root_files/less_overbias_0.root", "root_files/less_overbias_p10.root", "root_files/less_overbias_p20.root", "root_files/less_overbias_p30.root", "root_files/less_overbias_p40.root"] 
files4 = ["root_files/lin1_n40.root", "root_files/lin1_n30.root", "root_files/lin1_n20.root", "root_files/lin1_n10.root", "root_files/lin1_0.root", "root_files/lin1_p10.root", "root_files/lin1_p20.root", "root_files/lin1_p30.root", "root_files/lin1_p40.root"]
files5 = ["root_files/lin2_n40.root", "root_files/lin2_n30.root", "root_files/lin2_n20.root", "root_files/lin2_n10.root", "root_files/lin2_0.root", "root_files/lin2_p10.root", "root_files/lin2_p20.root", "root_files/lin2_p30.root", "root_files/lin2_p40.root"]
files6 = ["root_files/lin3_n40.root", "root_files/lin3_n30.root", "root_files/lin3_n20.root", "root_files/lin3_n10.root", "root_files/lin3_0.root", "root_files/lin3_p10.root", "root_files/lin3_p20.root", "root_files/lin3_p30.root", "root_files/lin3_p40.root"]
files7 = ["root_files/lin11_n40.root", "root_files/lin11_n30.root", "root_files/lin11_n20.root", "root_files/lin11_n10.root", "root_files/lin11_0.root", "root_files/lin11_p10.root", "root_files/lin11_p20.root", "root_files/lin11_p30.root", "root_files/lin11_p40.root"]
files8 = ["root_files/lin5_n40.root", "root_files/lin5_n30.root", "root_files/lin5_n20.root", "root_files/lin5_n10.root", "root_files/lin5_0.root", "root_files/lin5_p10.root", "root_files/lin5_p20.root", "root_files/lin5_p30.root", "root_files/lin5_p40.root"]
files9 = ["root_files/lin6_n40.root", "root_files/lin6_n30.root", "root_files/lin6_n20.root", "root_files/lin6_n10.root", "root_files/lin6_0.root", "root_files/lin6_p10.root", "root_files/lin6_p20.root", "root_files/lin6_p30.root", "root_files/lin6_p40.root"]
files10 = ["root_files/lin7_n40.root", "root_files/lin7_n30.root", "root_files/lin7_n20.root", "root_files/lin7_n10.root", "root_files/lin7_0.root", "root_files/lin7_p10.root", "root_files/lin7_p20.root", "root_files/lin7_p30.root", "root_files/lin7_p40.root"]
files11 = ["root_files/lin8_n40.root", "root_files/lin8_n30.root", "root_files/lin8_n20.root", "root_files/lin8_n10.root", "root_files/lin8_0.root", "root_files/lin8_p10.root", "root_files/lin8_p20.root", "root_files/lin8_p30.root", "root_files/lin8_p40.root"]
files12 = ["root_files/lin9_n40.root", "root_files/lin9_n30.root", "root_files/lin9_n20.root", "root_files/lin9_n10.root", "root_files/lin9_0.root", "root_files/lin9_p10.root", "root_files/lin9_p20.root", "root_files/lin9_p30.root", "root_files/lin9_p40.root"]
files13 = ["root_files/lin10_n40.root", "root_files/lin10_n30.root", "root_files/lin10_n20.root", "root_files/lin10_n10.root", "root_files/lin10_0.root", "root_files/lin10_p10.root", "root_files/lin10_p20.root", "root_files/lin10_p30.root", "root_files/lin10_p40.root"]
files14 = ["root_files/lin12_n40.root", "root_files/lin12_n30.root", "root_files/lin12_n20.root", "root_files/lin12_n10.root", "root_files/lin12_0.root", "root_files/lin12_p10.root", "root_files/lin12_p20.root", "root_files/lin12_p30.root", "root_files/lin12_p40.root"]


positions = [40, 30, 20, 10, 0, -10, -20, -30, -40] 
files_tot = [files1]+[files2]+[files4]+[files5]+[files6]+[files7]+[files8]+[files9]+[files10]+[files11]+[files12]+[files13]+[files14]+[['root_files/lin_273_2_40.root', 'root_files/lin_273_2_30.root', 'root_files/lin_273_2_20.root', 'root_files/lin_273_2_10.root', 'root_files/lin_273_2_0.root', 'root_files/lin_273_2_-10.root', 'root_files/lin_273_2_-20.root', 'root_files/lin_273_2_-30.root', 'root_files/lin_273_2_-40.root'], ['root_files/lin_273_3_40.root', 'root_files/lin_273_3_30.root', 'root_files/lin_273_3_20.root', 'root_files/lin_273_3_10.root', 'root_files/lin_273_3_0.root', 'root_files/lin_273_3_-10.root', 'root_files/lin_273_3_-20.root', 'root_files/lin_273_3_-30.root', 'root_files/lin_273_3_-40.root']]

print(files_tot)
x_tot, y_tot, pulse_tot = [], [], []
for f in files_tot:
    x_values, y_values, pulse_names = [], [], []
    for dat in f:
        pulse, x_dat, y_dat = collect_pulse_data(dat)
        pulse_names.append(pulse)
        x_values.append(np.array(x_dat))
        y_values.append(np.array(y_dat))
        
    x_tot.append(x_values)
    y_tot.append(y_values)
    pulse_tot.append(pulse_names)

data_tot = array_data(x_tot, y_tot, np.linspace(0.1,5,num=100))
print(data_tot)

e_field = collect_efield(files_tot, positions)
print(e_field)

In [ ]:
column_names = ['Current_pulse', 'E_field']

x_train = data_tot.astype("float32")
y_train = e_field.astype("float32")

def build_and_compile_model():
  model = keras.Sequential([
    layers.Dense(1000, activation='relu'),
    layers.Dense(1000, activation='relu'),
    layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

model = build_and_compile_model()

history = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    verbose=0, epochs=175)



In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='Loss')
  plt.plot(history.history['val_loss'], label='Validation Loss')
  #plt.ylim(0,1000)
  plt.title("Losses for the Machine Learning Model")
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)
  
plot_loss(history)
print(history.history['loss'])

In [ ]:
test_files = [["root_files/lin4_n40.root", "root_files/lin4_n30.root", "root_files/lin4_n20.root", "root_files/lin4_n10.root", "root_files/lin4_0.root", "root_files/lin4_p10.root", "root_files/lin4_p20.root", "root_files/lin4_p30.root", "root_files/lin4_p40.root"]]
x_test, y_test, pulse_test = [], [], []
for f in test_files:
    x_1, y_1, pulse_names = [], [], []
    for dat in f:
        pulse, x, y = collect_pulse_data(dat)
        pulse_names.append(pulse)
        x_1.append(np.array(x))
        y_1.append(np.array(y))
        
    x_test.append(x_1)
    y_test.append(y_1)
    pulse_test.append(pulse_names)

data_test = array_data(x_test, y_test, np.linspace(0.1,5,num=100))
#print(data_test)
y = model.predict(data_test)
print(y)

In [ ]:
print(y[0])
plt.plot(positions, collect_efield(test_files, positions)[0], label='Allpix Applied Electric Field')
plt.plot(positions, y[0], color='k', label='ML Prediction for the Electric Field')
plt.title("Electric Field Prediction using ML Model")
plt.xlabel('Position (\u03bcm)')
plt.ylabel('E_field (V/cm)')
#plt.ylim(-85000,-45000)
plt.legend()
plt.show()